In [ ]:
# importing nescessary libabries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

In [ ]:
# Load the CSV file
data_df = pd.read_csv('flowty_loan_indexer.csv')

# Removing invalid data from the csv , this usually happens with the last row of every csv you download from Flipside
data_df = data_df[data_df['LISTING_DATE'] != "\ufeff"]


In [ ]:
# Check the shape of the dataset
print(data_df.shape)

In [ ]:
# Get basic statistics
print(data_df.describe())

In [ ]:
# Check data types
print(data_df.dtypes)

In [ ]:
# Convert date-related columns to datetime format
date_columns = [
    'LISTING_DATE', 'LISTING_EXPIRY_DATE', 'FUNDING_DATE', 
    'LOAN_SETTLEMENT_DATE', 'LIQUDATION_DATE', 'REPAYMENT_DATE'
]

for column in date_columns:
    data_df[column] = pd.to_datetime(data_df[column])

# Verify the changes by checking the data types again
print(data_df.dtypes)

In [ ]:
# Function to check if a date string is valid
def is_valid_date(date_str):
    try:
        pd.to_datetime(date_str)
        return True
    except:
        return False

# Identify invalid date formats in each date column
invalid_date_rows = {}

for column in date_columns:
    invalid_rows = data_df[~data_df[column].apply(is_valid_date)]
    if not invalid_rows.empty:
        invalid_date_rows[column] = invalid_rows[[column]]

invalid_date_rows

In [ ]:
# Check for missing values
missing_values = data_df.isnull().sum()
print(missing_values)

In [ ]:
# Handling Funding-Related Columns
data_df.loc[:, 'FUNDING_DATE'].fillna('2099-01-01', inplace=True)  # Placeholder date
data_df.loc[:, 'LOAN_SETTLEMENT_DATE'].fillna('2099-01-01', inplace=True)
data_df.loc[:, ['FUNDINGRESOURCEID', 'EXPECTEDREPAYMENTAMOUNT', 'EXPECTED_REPAYMENT_AMOUNT_IN_USD']].fillna(-1, inplace=True)
data_df.loc[:, ['BORROWER', 'LENDER']].fillna('Unfunded', inplace=True)

# Handling Liquidation-Related Columns
data_df.loc[:, 'LIQUDATION_DATE'].fillna('2099-01-01', inplace=True) 
data_df.loc[:, ['LIQUIDATEDAMOUNT', 'LIQUIDATED_AMOUNT_IN_USD']].fillna(-1, inplace=True)

# Handling Repayment-Related Columns
data_df.loc[:, 'REPAYMENT_DATE'].fillna('2099-01-01', inplace=True)
data_df.loc[:, ['REPAIDAMOUNT', 'REPAID_AMOUNT_IN_USD']].fillna(-1, inplace=True)



In [ ]:
# Handling the non-date columns
columns_to_fill_with_minus_one = [
    'FUNDINGRESOURCEID', 'EXPECTEDREPAYMENTAMOUNT', 
    'EXPECTED_REPAYMENT_AMOUNT_IN_USD', 'LIQUIDATEDAMOUNT', 
    'LIQUIDATED_AMOUNT_IN_USD', 'REPAIDAMOUNT', 'REPAID_AMOUNT_IN_USD'
]

for column in columns_to_fill_with_minus_one:
    data_df[column].fillna(-1, inplace=True)

data_df['BORROWER'].fillna('Unfunded', inplace=True)
data_df['LENDER'].fillna('Unfunded', inplace=True)

# Check for missing values again
missing_values_updated = data_df.isnull().sum()
missing_values_updated

 Exploratory Data Analysis (EDA) and Visualization:

In [ ]:
data_summary = data_df.describe(include='all')
print(data_summary)

In [ ]:
# Distribution of Loan Amounts

sns.distplot(data_df['AMOUNT_IN_USD'], kde=True)
plt.title('Distribution of Loan Amounts in USD')
plt.show()


In [ ]:
# Since outliers were too much we did log transformation of the data
# Apply log transformation to the data
log_amount = np.log1p(data_df['AMOUNT_IN_USD'])  # log1p is used to handle zero values

sns.distplot(log_amount, kde=True)
plt.title('Log-transformed Distribution of Loan Amounts in USD')
plt.xlabel('Log(Amount in USD)')
plt.show()


In [ ]:
# Loan Status Distribution
status_counts = data_df['STATUS'].value_counts()
status_counts.plot(kind='bar')
plt.title('Loan Status Distribution')
plt.ylabel('Number of Loans')
plt.show()

In [ ]:

log_amount_ir = np.log1p(data_df['INTERESTRATE_IN_PERCENT'])  # log1p is used to handle zero values


sns.distplot(log_amount_ir, kde=True)
plt.title('Distribution of Interest Rates (%)')
plt.show()

In [ ]:
corr_columns = [
    'LISTING_DATE', 'ISFUNDED', 'ENABLEDAUTOREPAYMENT',  'AMOUNT', 'AMOUNT_IN_USD', 
    'INTERESTRATE_IN_PERCENT', 'ROYALTYRATE', 'TERM_IN_DAYS', 'EXPECTEDREPAYMENTAMOUNT', 
    'EXPECTED_REPAYMENT_AMOUNT_IN_USD', 'LIQUIDATED_AMOUNT_IN_USD', 
    'REPAIDAMOUNT',  'LIQUIDATEDAMOUNT', 'REPAID_AMOUNT_IN_USD'
]
correlation_matrix = data_df[corr_columns].corr()

sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show() 

In [ ]:
# Correlation Matrix with better formatting 
plt.figure(figsize=(15, 10))

# Mask for the upper triangle
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# Setting up the seaborn heatmap
sns.heatmap(correlation_matrix, 
            annot=True, 
            cmap='coolwarm', 
            fmt=".2f", 
            mask=mask, 
            linewidths=0.5, 
            cbar_kws={"shrink": 0.75})

plt.title('Correlation Heatmap', fontsize=15)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()


In [ ]:
# Example: Total loan amounts over time
time_series_data = data_df.groupby('LISTING_DATE')['AMOUNT_IN_USD'].sum()
time_series_data.plot()
plt.title('Total Loan Amounts Over Time')
plt.ylabel('Total Amount in USD')
plt.show()

In [ ]:
# Filter data for funded and unfunded loans
funded_loans = data_df[data_df['ISFUNDED'] == True]
unfunded_loans = data_df[data_df['ISFUNDED'] == False]

# Group by LISTING_DATE and sum the AMOUNT_IN_USD
time_series_funded = funded_loans.groupby('LISTING_DATE')['AMOUNT_IN_USD'].sum()
time_series_unfunded = unfunded_loans.groupby('LISTING_DATE')['AMOUNT_IN_USD'].sum()

# Plot
plt.figure(figsize=(12, 6))
time_series_funded.plot(label='Funded Loans')
time_series_unfunded.plot(label='Unfunded Loans')
plt.title('Total Loan Amounts Over Time')
plt.ylabel('Total Amount in USD')
plt.xlabel('Listing Date')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Check for missing values
missing_values = data_df.isnull().sum()
print(missing_values)